In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
import underthesea


In [2]:
pip install unidecode

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
import re
from unidecode import unidecode
import underthesea
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib
import numpy as np

In [4]:
df  = pd.read_csv('cafef_full_data_2.csv')

In [5]:
df.head()

,category,title,link,description,time_posted,crawl_date,content
0,doanh-nghiep,Một công ty lãi gấp 36 lần nhờ chốt lời cổ phi...,https://cafef.vn/mot-cong-ty-lai-gap-36-lan-nh...,NDN: 12.6 | -0.2 (-1.6%)\nĐây là kết quả cao n...,NaN,2025-10-24 11:07:02,CTCP Đầu tư Phát triển Nhà Đà Nẵng (mã CK: NDN...
1,doanh-nghiep,'Gã khổng lồ' đóng tàu của Samsung dự kiến đón...,https://cafef.vn/ga-khong-lo-dong-tau-cua-sams...,1 giờ trước\n3 tàu này dự kiến bàn giao vào th...,NaN,2025-10-24 11:07:02,"Theo The Korea Times, Samsung Heavy Industrie..."
2,doanh-nghiep,"Shark Phú lập 'đội đặc vụ' bán nồi, chảo trên ...",https://cafef.vn/shark-phu-lap-doi-dac-vu-ban-...,2 giờ trước\nMục tiêu của Sunhouse là đạt doan...,NaN,2025-10-24 11:07:02,Ông Nguyễn Xuân Phú - Chủ tịch HĐQT Tập đoàn ...
3,doanh-nghiep,DN muốn làm dự án bệnh viện 1.500 tỷ tại Nghệ ...,https://cafef.vn/dn-muon-lam-du-an-benh-vien-1...,MST: 5.5 | -0.1 (-1.8%)\nĐầu tư MST cũng đã mu...,NaN,2025-10-24 11:07:02,Dự án bệnh viện 1.500 tỷ tại Nghệ An Theo thôn...
4,doanh-nghiep,Cập nhật BCTC quý 3/2025 sáng ngày 24/10: Công...,https://cafef.vn/cap-nhat-bctc-quy-3-2025-sang...,10 giờ trước\nPhốt Pho Apatit (PAT) báo lãi tr...,NaN,2025-10-24 11:07:02,Các doanh nghiệp mới công bố BCTC quý 3/2025 n...


In [6]:
df = df[["category", "title", "content"]].dropna(subset=["title", "content"]).reset_index(drop=True)

In [7]:
def clean_vietnamese_text(text):
    if not isinstance(text, str):
        return ""
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)
    text = re.sub(r"\S+@\S+", "", text)
    text = re.sub(r"\d+", "", text)
    text = re.sub(r"[^a-zA-Z0-9\sáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđĐ]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text.lower()

In [8]:
# ======== 3️⃣ Tách mã cổ phiếu ==========
def extract_stock_codes(text):
    return list(set(re.findall(r"\b[A-Z]{3,3}\b", text)))

df["stock"] = df["title"].apply(extract_stock_codes) + df["content"].apply(extract_stock_codes)

In [9]:
df.head()

,category,title,content,stock
0,doanh-nghiep,Một công ty lãi gấp 36 lần nhờ chốt lời cổ phi...,CTCP Đầu tư Phát triển Nhà Đà Nẵng (mã CK: NDN...,"[VCG, VHM, NDN, GMD, IDC]"
1,doanh-nghiep,'Gã khổng lồ' đóng tàu của Samsung dự kiến đón...,"Theo The Korea Times, Samsung Heavy Industrie...","[USD, USD, SHI, LNG, DWT]"
2,doanh-nghiep,"Shark Phú lập 'đội đặc vụ' bán nồi, chảo trên ...",Ông Nguyễn Xuân Phú - Chủ tịch HĐQT Tập đoàn ...,[USD]
3,doanh-nghiep,DN muốn làm dự án bệnh viện 1.500 tỷ tại Nghệ ...,Dự án bệnh viện 1.500 tỷ tại Nghệ An Theo thôn...,"[EPC, USD, VAT, MST]"
4,doanh-nghiep,Cập nhật BCTC quý 3/2025 sáng ngày 24/10: Công...,Các doanh nghiệp mới công bố BCTC quý 3/2025 n...,"[IDV, VHD, EVN, PAT]"


In [36]:
# ======== 4️⃣ Tách tên công ty ==========
COMPANY_KEYWORDS = [
    "vin", "vingroup", "vietcombank", "fpt", "vietinbank", "bidv",
    "hoa phat", "vietjet", "masan", "samsung", "vietnam airlines",
    "vib", "acb", "mbbank", "shb", "techcombank", "vpbank"
]
def extract_company_names(text):
    t = unidecode(text.lower())
    return [kw for kw in COMPANY_KEYWORDS if kw in t]

df["company"] = df["title"].apply(extract_company_names) + df["content"].apply(extract_company_names)

In [37]:
df.head()

,category,title,content,stock,company
0,doanh-nghiep,Một công ty lãi gấp 36 lần nhờ chốt lời cổ phi...,CTCP Đầu tư Phát triển Nhà Đà Nẵng (mã CK: NDN...,"[GMD, NDN, IDC, VCG, VHM]","[vin, vin]"
1,doanh-nghiep,'Gã khổng lồ' đóng tàu của Samsung dự kiến đón...,"Theo The Korea Times, Samsung Heavy Industrie...","[USD, USD, SHI, DWT, LNG]","[samsung, samsung]"
2,doanh-nghiep,"Shark Phú lập 'đội đặc vụ' bán nồi, chảo trên ...",Ông Nguyễn Xuân Phú - Chủ tịch HĐQT Tập đoàn ...,[USD],[]
3,doanh-nghiep,DN muốn làm dự án bệnh viện 1.500 tỷ tại Nghệ ...,Dự án bệnh viện 1.500 tỷ tại Nghệ An Theo thôn...,"[EPC, MST, USD, VAT]","[vin, vietinbank]"
4,doanh-nghiep,Cập nhật BCTC quý 3/2025 sáng ngày 24/10: Công...,Các doanh nghiệp mới công bố BCTC quý 3/2025 n...,"[EVN, VHD, PAT, IDV]",[vin]


In [ ]:




# ======== 5️⃣ Mapping company -> stock ==========
COMPANY_STOCK_MAP = {
    "vingroup": "VIC",
    "vietcombank": "VCB",
    "fpt": "FPT",
    "vietinbank": "CTG",
    "bidv": "BID",
    "hoa phat": "HPG",
    "vietjet": "VJC",
    "masan": "MSN",
    "vib": "VIB",
    "acb": "ACB",
    "mbbank": "MBB",
    "shb": "SHB",
    "techcombank": "TCB",
    "vpbank": "VPB"
}
def map_company_to_stock(companies):
    if not companies:
        return []
    result = []
    for c in companies:
        if c in COMPANY_STOCK_MAP:
            result.append(COMPANY_STOCK_MAP[c])
    return list(set(result))

df["mapped_stock"] = df["company"].apply(map_company_to_stock)

# ======== 6️⃣ Stopword removal ==========
STOPWORDS = set([
    "và", "là", "của", "có", "cho", "với", "được", "này", "đã", "trong", "khi",
    "tại", "một", "các", "những", "đến", "để", "về", "ra", "thì", "cũng", "như",
    "rằng", "theo", "từ", "vào", "nên", "sẽ", "đang", "vì", "nếu", "hơn"
])
def remove_stopwords(text):
    return " ".join([t for t in text.split() if t not in STOPWORDS])

df["title_nostop"] = df["title_clean"].progress_apply(remove_stopwords)
df["content_nostop"] = df["content_clean"].progress_apply(remove_stopwords)

# ======== 7️⃣ Tokenization ==========
def tokenize_vietnamese(text):
    try:
        return underthesea.word_tokenize(text, format="text")
    except:
        return text

df["title_tok"] = df["title_nostop"].progress_apply(tokenize_vietnamese)
df["content_tok"] = df["content_nostop"].progress_apply(tokenize_vietnamese)

# ======== 8️⃣ Không dấu ==========
df["title_noaccent"] = df["title_clean"].apply(unidecode)
df["content_noaccent"] = df["content_clean"].apply(unidecode)

# ======== 9️⃣ TF-IDF Vectorization ==========
def build_tfidf(series, max_features=3000):
    vectorizer = TfidfVectorizer(
        max_features=max_features,
        ngram_range=(1, 2),
        min_df=2,
        sublinear_tf=True
    )
    tfidf_matrix = vectorizer.fit_transform(series)
    return vectorizer, tfidf_matrix

print("\n⚙️ Đang tạo TF-IDF cho title ...")
tfidf_title_vectorizer, tfidf_title = build_tfidf(df["title_tok"])

print("⚙️ Đang tạo TF-IDF cho content ...")
tfidf_content_vectorizer, tfidf_content = build_tfidf(df["content_tok"])

# Lưu mô hình TF-IDF
joblib.dump(tfidf_title_vectorizer, "tfidf_title.pkl")
joblib.dump(tfidf_content_vectorizer, "tfidf_content.pkl")
np.savez_compressed("tfidf_vectors.npz", title=tfidf_title, content=tfidf_content)

# ======== 🔟 Lưu CSV hoàn chỉnh ==========
df.to_csv("cafef_preprocessed_full.csv", index=False, encoding="utf-8-sig")

print("\n✅ Preprocessing hoàn tất!")
print(f"📁 Dữ liệu sạch: cafef_preprocessed_full.csv")
print(f"📦 TF-IDF lưu tại: tfidf_title.pkl, tfidf_content.pkl, tfidf_vectors.npz")